#### Libraries

In [2]:
import pandas as pd
import numpy as np
import glob
import os

import warnings
warnings.filterwarnings('ignore')

#### Importing Data

In [2]:
# Data specs #
data_header_col_specs = [(0,11),(11,15),(15,17),(17,21)]
data_header_names = ["ID","YEAR","MONTH","ELEMENT"]
data_header_dtypes = {"ID":str, "YEAR":int, "MONTH":int, "ELEMENT":str}

data_col_names = [["VALUE" + str(i+1),
                  "MFLAG" + str(i+1),
                  "QFLAG" + str(i+1),
                  "SFLAG" + str(i+1)]
                 for i in range(31)]
data_col_names = sum(data_col_names, [])

data_col_specs = [[
    (21 + i * 8, 26 + i * 8),
    (26 + i * 8, 27 + i * 8),
    (27 + i * 8, 28 + i * 8),
    (28 + i * 8, 29 + i * 8)]
    for i in range(31)]
data_col_specs = sum(data_col_specs, [])

data_col_dtypes = [{
    "VALUE" + str(i + 1): int,
    "MFLAG" + str(i + 1): str,
    "QFLAG" + str(i + 1): str,
    "SFLAG" + str(i + 1): str}
    for i in range(31)]
data_header_dtypes.update({k: v for d in data_col_dtypes for k, v in d.items()})

vals = ["VALUE" + str(i+1) for i in range(31)]
mfs = ["MFLAG" + str(i+1) for i in range(31)]
qfs = ["QFLAG" + str(i+1) for i in range(31)]
sfs = ["SFLAG" + str(i+1) for i in range(31)]
day = [i+1 for i in range(31)]

In [3]:
def read_ghcn_data_file(filename="USC00011084.dly"):
    """Reads in all data from a GHCN .dly data file

    :param filename: path to file
    :param variables: list of variables to include in output dataframe
        e.g. ['TMAX', 'TMIN', 'PRCP']
    :param include_flags: Whether to include data quality flags in the final output
    :returns: Pandas dataframe
    """

    df = pd.read_fwf(
        filename,
        colspecs=data_header_col_specs + data_col_specs,
        names=data_header_names + data_col_names,
        dtype=data_header_dtypes
        )
    
    return df

In [14]:
#Getting all files from directory and importing
path = '/Users/hkelderman/DataScience/Bootcamp/Capstone/Climate Data/ghcnd_hcn'
all_files = glob.glob(path + "/*.dly")

li = []

for filename in all_files:
    df = read_ghcn_data_file(filename)
    li.append(df)

In [15]:
#concatonating all all climate dataframes into one
frame = pd.concat(li, axis=0, ignore_index=True)

In [17]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9663559 entries, 0 to 9663558
Columns: 128 entries, ID to SFLAG31
dtypes: int64(33), object(95)
memory usage: 9.2+ GB


####  Saving Needed Data

In [18]:
frame.head()

,ID,YEAR,MONTH,ELEMENT,VALUE1,MFLAG1,QFLAG1,SFLAG1,VALUE2,MFLAG2,...,QFLAG29,SFLAG29,VALUE30,MFLAG30,QFLAG30,SFLAG30,VALUE31,MFLAG31,QFLAG31,SFLAG31
0,USC00137147,1893,4,TMAX,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,0,67,NaN,NaN,0,-9999,NaN,NaN,NaN
1,USC00137147,1893,4,TMIN,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,0,22,NaN,NaN,0,-9999,NaN,NaN,NaN
2,USC00137147,1893,4,PRCP,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,0,0,T,NaN,0,-9999,NaN,NaN,NaN
3,USC00137147,1893,4,SNOW,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,0,0,NaN,NaN,0,-9999,NaN,NaN,NaN
4,USC00137147,1893,4,SNWD,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN


In [19]:
df = frame[frame.YEAR >= 1984]

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3486814 entries, 6899 to 9663558
Columns: 128 entries, ID to SFLAG31
dtypes: int64(33), object(95)
memory usage: 3.4+ GB


In [40]:
#Splitting the data into more managable dataframes
df1 = df.iloc[0:871704]
df2 = df.iloc[871704:1743408]
df3 = df.iloc[1743408:2615112]
df4 = df.iloc[2615112:3486814]

In [41]:
#then saving the split dataframes to csvs, so as to not have to go through the joining process again
df1.to_csv('Climate_Data_1.csv')
df2.to_csv('Climate_Data_2.csv')
df3.to_csv('Climate_Data_3.csv')
df4.to_csv('Climate_Data_4.csv')

#### Cleaning Data

In [5]:
#Reading in CSVs after Kernal restart
df1 = pd.read_csv('Climate_Data_1.csv')
df2 = pd.read_csv('Climate_Data_2.csv')
df3 = pd.read_csv('Climate_Data_3.csv')
df4 = pd.read_csv('Climate_Data_4.csv')

In [6]:
#Lists for climate data cleaning
elements_to_keep = ['TMAX', 'TMIN', 'PRCP', 'AWND']
data_header_names = ["ID","YEAR","MONTH","ELEMENT"]
vals = ["VALUE" + str(i+1) for i in range(31)]

In [7]:
def climate_clean_1(df):
    """This function will clean each dataframe of daily climate data. It will remove elements we don't care about,
    melt the dataframe so that we have one row per day of values, and create a date column we may use further on.
    
    Attributes:
    df - dataframe of daily climate data
    """
    
    df = df[df.ELEMENT.isin(elements_to_keep)] #remove excess elements
    df_2 = pd.melt(df, id_vars=data_header_names, value_vars=vals, var_name='Day_col') #elongate dataframe
    df_2['Day'] = df_2['Day_col'].str.replace('VALUE', '') #create day column based on integer in 'Day_col'
    df_2[['YEAR', 'MONTH', 'Day']] = df_2[['YEAR', 'MONTH', 'Day']].astype(str) #convert each piece of date to str
    df_2['Day_clean'] = np.where(len(df_2['Day']) == 2, df_2['Day'], df_2['Day'].apply(lambda x: x.rjust(2, '0'))) #pad zeros to day
    df_2['Month_clean'] = np.where(len(df_2['MONTH']) == 2, df_2['MONTH'], df_2['MONTH'].apply(lambda x: x.rjust(2, '0'))) #pad zeros to month
    df_2['Date'] = df_2[['YEAR', 'Month_clean', 'Day_clean']].apply(lambda x: '-'.join(x), axis=1) #join to create a date col
    df_2['Date'] = pd.to_datetime(df_2['Date'], format='%Y-%m-%d', errors='coerce')
    
    df_2.drop(columns=['Day_col', 'YEAR', 'MONTH', 'Day', 'Day_clean', 'Month_clean'], inplace=True) #remove excess columns
    df_2['value'] = np.where((df_2['value'] == -9999), np.nan, df_2['value']) #replacing -9999 with NaN, so as not to interfere with future calculations
    df_2.dropna(subset=['Date'], inplace=True)
    
    return df_2

In [8]:
def climate_clean_2(df):
    """Now that we've removed the excess elements, created a date column, and taken care of the missing values,
    we will use this function to spread the dataframe back out based on the elements, as well as calculate the true
    value for each element based on the text found in the NOAA ReadMe file. We will also use this function to create
    various rolling averages for each date.
    
    Attributes:
    df - dataframe of daily climate data cleaned through round one
    """
    #spread data out based on ELEMENT column
    df = df.pivot(index=['ID', 'Date'], columns='ELEMENT', values='value')
    df.reset_index(inplace=True)
    
    #Temperature is reported tenths of degrees C, so this will convert the temperatures to degrees C
    df['max_temp_C'] = df['TMAX'] / 10
    df['min_temp_C'] = df['TMIN'] / 10

    #Precipitation is reported in tenths of mm, so this will convert the precipiation to cm
    df['precipitation_cm'] = df['PRCP'] / 100

    #Wind speed is measured in tenths of a meter/s, so this will convert the speed to m/s
    df['wind_speed_ms'] = df['AWND'] / 10
    
    #drop extra columns
    df.drop(columns=['AWND', 'PRCP', 'TMAX', 'TMIN'], inplace=True)
    
    #creating rolling averages
    cols = ['max_temp_C', 'min_temp_C', 'precipitation_cm', 'wind_speed_ms']
    periods = [30, 90, 180, 365]
    period_str = ['30', '90', '180', '365']
    
    for col in cols:
        for i in range(len(periods)):
            df['{}_{}'.format(col,period_str[i])] = df.groupby('ID')[col].transform(lambda x: x.rolling(periods[i], 1).mean())
        
    return df

In [9]:
daily_1 = climate_clean_1(df1)
daily_2 = climate_clean_1(df2)
daily_3 = climate_clean_1(df3)
daily_4 = climate_clean_1(df4)

In [10]:
daily_1 = climate_clean_2(daily_1)
daily_2 = climate_clean_2(daily_2)
daily_3 = climate_clean_2(daily_3)
daily_4 = climate_clean_2(daily_4)

In [9]:
#Combining
daily_weather = pd.concat([daily_1, daily_2, daily_3, daily_4])

### Selecting only dates of fires

In [10]:
fire_dates = pd.read_csv('Fire_dates.csv')
dates = list(fire_dates.IG_Date.unique())

In [11]:
len(daily_weather)

14603596

In [12]:
weather = daily_weather[daily_weather['Date'].isin(dates)]

In [13]:
len(weather)

7211938

### Join with meta-data

In [14]:
mon_reg = pd.read_csv('Monitor_by_Region.csv')
mon_reg.drop(columns='Unnamed: 0', inplace= True)

In [15]:
weather_mon = pd.merge(weather, mon_reg, how='left', on='ID')

In [16]:
weather_mon.head()

,ID,Date,max_temp_C,min_temp_C,precipitation_cm,wind_speed_ms,max_temp_C_30,max_temp_C_90,max_temp_C_180,max_temp_C_365,...,wind_speed_ms_90,wind_speed_ms_180,wind_speed_ms_365,LATITUDE,LONGITUDE,ELEVATION,STATE,NA_L3NAME,NA_L2NAME,NA_L1NAME
0,USC00013816,1984-02-21,15.6,6.1,0.05,NaN,16.493333,14.515385,14.515385,14.515385,...,NaN,NaN,NaN,31.8814,-86.2503,132.0,AL,Southeastern Plains,SOUTHEASTERN USA PLAINS,EASTERN TEMPERATE FORESTS
1,USC00013816,1984-03-29,27.2,7.8,0.05,NaN,19.923333,16.564045,16.564045,16.564045,...,NaN,NaN,NaN,31.8814,-86.2503,132.0,AL,Southeastern Plains,SOUTHEASTERN USA PLAINS,EASTERN TEMPERATE FORESTS
2,USC00013816,1984-04-18,18.3,6.7,0.00,NaN,21.730000,18.252222,17.354128,17.354128,...,NaN,NaN,NaN,31.8814,-86.2503,132.0,AL,Southeastern Plains,SOUTHEASTERN USA PLAINS,EASTERN TEMPERATE FORESTS
3,USC00013816,1984-04-20,24.4,15.6,0.23,NaN,21.766667,18.695556,17.461261,17.461261,...,NaN,NaN,NaN,31.8814,-86.2503,132.0,AL,Southeastern Plains,SOUTHEASTERN USA PLAINS,EASTERN TEMPERATE FORESTS
4,USC00013816,1984-04-21,29.4,17.2,0.00,NaN,22.116667,18.973333,17.567857,17.567857,...,NaN,NaN,NaN,31.8814,-86.2503,132.0,AL,Southeastern Plains,SOUTHEASTERN USA PLAINS,EASTERN TEMPERATE FORESTS


### Creating Summary Data by Date and Region

In [17]:
weather_mon.columns

Index(['ID', 'Date', 'max_temp_C', 'min_temp_C', 'precipitation_cm',
       'wind_speed_ms', 'max_temp_C_30', 'max_temp_C_90', 'max_temp_C_180',
       'max_temp_C_365', 'min_temp_C_30', 'min_temp_C_90', 'min_temp_C_180',
       'min_temp_C_365', 'precipitation_cm_30', 'precipitation_cm_90',
       'precipitation_cm_180', 'precipitation_cm_365', 'wind_speed_ms_30',
       'wind_speed_ms_90', 'wind_speed_ms_180', 'wind_speed_ms_365',
       'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NA_L3NAME', 'NA_L2NAME',
       'NA_L1NAME'],
      dtype='object')

In [18]:
#creating dictionary for aggregate functions
f = {'max_temp_C_30': ['mean'], 'max_temp_C_90': ['mean'], 'max_temp_C_180': ['mean'],
       'max_temp_C_365': ['mean'], 'min_temp_C_30': ['mean'], 'min_temp_C_90': ['mean'], 'min_temp_C_180': ['mean'],
       'min_temp_C_365': ['mean'], 'precipitation_cm_30': ['sum'], 'precipitation_cm_90': ['sum'],
       'precipitation_cm_180': ['sum'], 'precipitation_cm_365': ['sum'], 'wind_speed_ms_30': ['mean'],
       'wind_speed_ms_90': ['mean'], 'wind_speed_ms_180': ['mean'], 'wind_speed_ms_365': ['mean'],}

In [19]:
#creating average weather statistics for ecoregion level 1
level_1_weather = weather_mon.groupby(['Date', 'NA_L1NAME']).agg(f)
level_1_weather.reset_index(inplace=True)

In [20]:
#creating average weather statistics for ecoregion level 2
level_2_weather = weather_mon.groupby(['Date', 'NA_L2NAME']).agg(f)
level_2_weather.reset_index(inplace=True)

In [21]:
#creating average weather statistics for ecoregion level 3
level_3_weather = weather_mon.groupby(['Date', 'NA_L3NAME']).agg(f)
level_3_weather.reset_index(inplace=True)

In [22]:
#Writing CSVs to pull into MTBS analysis
level_1_weather.to_csv('Ecoregion Level 1 Weather.csv')
level_2_weather.to_csv('Ecoregion Level 2 Weather.csv')
level_3_weather.to_csv('Ecoregion Level 3 Weather.csv')